In [ ]:
SELECT current_database() AS DATABASE_NAME, current_schema() AS SCHEMA_NAME

In [ ]:
# Import python packages
import logging
# from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

logger = logging.getLogger("fred_logger")

current_context_df = cells.sql_get_context.to_pandas()
database_name = current_context_df.iloc[0,0]
schema_name = current_context_df.iloc[0,1]

session = get_active_session()

logger.info("03_analytics_table_processing start")

In [ ]:
CREATE OR REPLACE PROCEDURE merge_fred_updates_sp(DATABASE_NAME STRING, SCHEMA_NAME STRING)
 RETURNS STRING
 LANGUAGE PYTHON
 RUNTIME_VERSION=3.9
 PACKAGES=('snowflake-snowpark-python')
 HANDLER='main'
AS
$$
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F

env = schema_name[:3]

def table_exists(session, schema='', name=''):
    exists = session.sql("SELECT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{}' AND TABLE_NAME = '{}') AS TABLE_EXISTS".format(schema, name)).collect()[0]['TABLE_EXISTS']
    return exists

def create_fred_table(session):
    _ = session.sql(f"CREATE TABLE FRED_10Y_2Y LIKE {DATABASE_NAME}.{env}_HARMONIZED.FRED_FLATTENED").collect()
    _ = session.sql("ALTER TABLE FRED_10Y_2Y ADD COLUMN META_UPDATED_AT TIMESTAMP").collect()

# Uncomment only if we need to process another table
# def create_fred_stream(session):
#     _ = session.sql("CREATE STREAM FRED_10Y_2Y_STREAM ON TABLE FRED_10Y_2Y").collect()

def merge_fred_updates(session):
    # _ = session.sql('ALTER WAREHOUSE HOL_WH SET WAREHOUSE_SIZE = XLARGE WAIT_FOR_COMPLETION = TRUE').collect()

    source = f"{DATABASE_NAME}.{env}_HARMONIZED.FRED_STREAM"
    target = f"{DATABASE_NAME}.{SCHEMA_NAME}.FRED_10Y_2Y"

    # TODO: Is the if clause supposed to be based on "META_UPDATED_AT"?
    cols_to_update = {c: source[c] for c in source.schema.names if "METADATA" not in c}
    metadata_col_to_update = {"META_UPDATED_AT": F.current_timestamp()}
    updates = {**cols_to_update, **metadata_col_to_update}

    # merge into DIM_CUSTOMER
    target.merge(source, target['OBSERVATION_DATE'] == source['OBSERVATION_DATE'], \
                        [F.when_matched().update(updates), F.when_not_matched().insert(updates)])
    
def main(session: Session, DATABASE_NAME: str, SCHEMA_NAME: str) -> str:
    
    if not table_exists(session, schema=SCHEMA_NAME, name='FRED_10Y_2Y'):
            create_fred_table(session)
            # create_fred_stream(session)
    # Process data incrementally
    merge_fred_updates(session)
    return "FRED_10Y_2Y table updated successfully!"

$$;


In [ ]:
session.use_schema(f"{database_name}.{schema_name}")
result = session.sql(f"CALL merge_fred_updates_sp('{database_name}', '{schema_name}')").collect()

# Print result
print(result[0][0])  
SELECT 'SUNDAY', 0.9;